In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/Users/estyle-150/github/my_signage/data/processed/processed20240826_2.csv')
df

,datetime,y,client,close,price_am,price_pm,is_train,price,y_lag_365,price_am_lag_1,price_am_lag_7,price_am_lag_365,price_pm_lag_1,price_pm_lag_7,price_pm_lag_365,price_am_moving_avg_7,price_am_moving_avg_30,price_pm_moving_avg_7,price_pm_moving_avg_30
0,2011-01-04,16,0,0,0.0,0.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,16,0,0,0.0,0.0,1,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,13,0,0,0.0,0.0,1,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,14,0,0,0.0,0.0,1,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-08,16,0,0,0.0,0.0,1,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,2017-03-27,0,0,0,5.0,4.0,0,4.5,94.0,4.0,3.0,5.0,4.0,4.0,4.0,3.857143,2.166667,4.000000,2.166667
2275,2017-03-28,0,0,0,4.0,4.0,0,4.0,86.0,5.0,4.0,4.0,4.0,4.0,4.0,3.857143,2.233333,4.000000,2.233333
2276,2017-03-29,0,0,0,3.0,3.0,0,3.0,98.0,4.0,2.0,4.0,4.0,4.0,4.0,4.000000,2.266667,3.857143,2.266667
2277,2017-03-30,0,0,0,4.0,3.0,0,3.5,99.0,3.0,3.0,5.0,3.0,2.0,4.0,4.142857,2.333333,4.000000,2.300000


In [4]:
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb_op
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error as mae

In [9]:
train_data = df[df['is_train']==1].copy()
train_data.drop(['is_train'], axis=1, inplace=True)

df_train, df_val = train_test_split(train_data,test_size=0.2, shuffle=True, random_state=9)

train_y = df_train['y']
train_x = df_train.drop('y', axis=1)

val_y = df_val['y']
val_x = df_val.drop('y',axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
  'objective':'regression',
  'metric': 'mae',
  'force_row_wise' : True,
  'verbose': -1
}


gbm = lgb.train(
    params,
    trains,
    valid_sets=valids,
    num_boost_round=10,
    verbose_eval=5,
)

TypeError: train() got an unexpected keyword argument 'verbose_eval'